In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")

In [2]:
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from sklearn.metrics import log_loss

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [3]:
train_df=pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
test_data=pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')

sample_submission=pd.read_csv('/kaggle/input/playground-series-s4e1/sample_submission.csv')

orig_df=pd.read_csv('/kaggle/input/bank-customer-churn-prediction/Churn_Modelling.csv').rename(columns={'RowNumber':'id'})

In [4]:
orig_df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,9998,15584532,Liu,709,France,Female,36.0,7,0.00,1,0.0,1.0,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9999,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
10000,10000,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0


In [5]:
train_df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165029,165029,15667085,Meng,667,Spain,Female,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,165030,15665521,Okechukwu,792,France,Male,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,165031,15664752,Hsia,565,France,Male,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,165032,15689614,Hsiung,554,Spain,Female,30.0,7,161533.00,1,0.0,1.0,71173.03,0


In [6]:
test_data

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110018,275052,15662091,P'eng,570,Spain,Male,29.0,7,116099.82,1,1.0,1.0,148087.62
110019,275053,15774133,Cox,575,France,Female,36.0,4,178032.53,1,1.0,1.0,42181.68
110020,275054,15728456,Ch'iu,712,France,Male,31.0,2,0.00,2,1.0,0.0,16287.38
110021,275055,15687541,Yegorova,709,France,Female,32.0,3,0.00,1,1.0,1.0,158816.58


In [7]:
pd.concat([train_df,orig_df]).duplicated().sum() #only two duplicated items

2

In [8]:
train_df=pd.concat([train_df,orig_df])
train_df.drop(['id'],axis=1,inplace=True)
train_df.shape

(175036, 13)

In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175036 entries, 0 to 10001
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CustomerId       175036 non-null  int64  
 1   Surname          175036 non-null  object 
 2   CreditScore      175036 non-null  int64  
 3   Geography        175035 non-null  object 
 4   Gender           175036 non-null  object 
 5   Age              175035 non-null  float64
 6   Tenure           175036 non-null  int64  
 7   Balance          175036 non-null  float64
 8   NumOfProducts    175036 non-null  int64  
 9   HasCrCard        175035 non-null  float64
 10  IsActiveMember   175035 non-null  float64
 11  EstimatedSalary  175036 non-null  float64
 12  Exited           175036 non-null  int64  
dtypes: float64(5), int64(5), object(3)
memory usage: 18.7+ MB


In [10]:
train_df.select_dtypes(include='object').nunique()

Surname      2932
Geography       3
Gender          2
dtype: int64

In [11]:
train_df.drop(['Surname','CustomerId'],axis=1,inplace=True)
train_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [12]:
test_df = test_data.drop(['id','Surname','CustomerId'],axis=1)
test_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [13]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())

CreditScore        0
Geography          1
Gender             0
Age                1
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          1
IsActiveMember     1
EstimatedSalary    0
Exited             0
dtype: int64
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64


In [14]:
train_df.dropna(inplace=True)

In [15]:
numerical_cols = test_df.select_dtypes(include=['int', 'float']).columns.tolist()
numerical_cols

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [16]:
categorical_cols = test_df.select_dtypes(include='object').columns.tolist()
categorical_cols

['Geography', 'Gender']

# Modeling

In [17]:
my_cols = categorical_cols + numerical_cols

X = train_df[my_cols].copy()
y = train_df.Exited

test_df = test_df[my_cols].copy()

In [18]:
from sklearn.model_selection import train_test_split

# Get 70% of the dataset as the training set. Put the remaining 30% in temporary variables: x_ and y_.
x_train, x_, y_train, y_ = train_test_split(X, y, test_size=0.30, random_state=1)

# Split the 30% subset above into two: one half for cross validation and the other for the test set
x_valid, x_test, y_valid, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

# Delete temporary variables
del x_, y_

print(f"the shape of the training set (input) is: {x_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x_valid.shape}")
print(f"the shape of the cross validation set (target) is: {y_valid.shape}\n")
print(f"the shape of the test set (input) is: {x_test.shape}")
print(f"the shape of the test set (target) is: {y_test.shape}")

the shape of the training set (input) is: (122522, 10)
the shape of the training set (target) is: (122522,)

the shape of the cross validation set (input) is: (26255, 10)
the shape of the cross validation set (target) is: (26255,)

the shape of the test set (input) is: (26255, 10)
the shape of the test set (target) is: (26255,)


### Pipeline

In [19]:
numerical_transformer = Pipeline(steps=[
    ('scalar', StandardScaler()),
    ('MeanImputer', SimpleImputer())
    ])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [20]:
new_X_train = pd.DataFrame(preprocessor.fit_transform(x_train))
new_X_valid = pd.DataFrame(preprocessor.transform(x_valid))
new_X_test = pd.DataFrame(preprocessor.transform(x_test))

new_test = pd.DataFrame(preprocessor.transform(test_df))

print(new_X_train.shape)
print(new_X_valid.shape)
print(new_X_test.shape)
print(new_test.shape)

(122522, 13)
(26255, 13)
(26255, 13)
(110023, 13)


### XGB Classifier

In [21]:
XGB_params = {'booster': 'gbtree', 
        'lambda': 8.803747805736341e-08, 
        'alpha': 0.00010339545129413926,
        'subsample': 0.6472067107394853,
        'colsample_bytree': 0.9225973102702412,
        'max_depth': 5,
        'min_child_weight': 5,
        'eta': 0.12477845463556776,
        'gamma': 0.0021228439224434607,
        'grow_policy': 'lossguide'}

XGB=XGBClassifier(**XGB_params)
XGB.fit(new_X_train,y_train)

XGBClassifier(alpha=0.00010339545129413926, base_score=None, booster='gbtree',
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9225973102702412, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eta=0.12477845463556776, eval_metric=None, feature_types=None,
              gamma=0.0021228439224434607, grow_policy='lossguide',
              importance_type=None, interaction_constraints=None,
              lambda=8.803747805736341e-08, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, ...)

In [22]:
pred_val_XGB=XGB.predict_proba(new_X_valid)
pred_val_XGB=[i[1] for i in pred_val_XGB]
print("XGB ROC validation score =",roc_auc_score(y_valid,pred_val_XGB))

XGB ROC validation score = 0.8859637976244653


In [23]:
pred_test_XGB=XGB.predict_proba(new_X_test)
pred_test_XGB=[i[1] for i in pred_test_XGB]
print("XGB ROC test score =",roc_auc_score(y_valid,pred_test_XGB))

XGB ROC test score = 0.50596541100621


# Prediction

In [24]:
y_pred=XGB.predict_proba(new_test)
y_pred=[i[1] for i in y_pred]
len(y_pred)

110023

In [25]:
output = pd.DataFrame({
    "id": test_data["id"].to_list(),
    "Exited": y_pred
})

output

,id,Exited
0,165034,0.027792
1,165035,0.817946
2,165036,0.025760
3,165037,0.219232
4,165038,0.360446
...,...,...
110018,275052,0.042926
110019,275053,0.184678
110020,275054,0.018752
110021,275055,0.128635


In [26]:
output.to_csv('submission.csv', index=False)